In [1]:
#CONDA ENV base_conda (python 3.9.7)
#Import packages
#---------------------------------------
import sys
import os
import glob
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm
import scanpy as sc
#Import your modules
#---------------------------------------
sys.path.insert(1, '/cndd3/dburrows/CODE/admin_tools/')
from admin_tools import admin_functions as adm

# Define paths
#----------------------------------------------------------------------
l_code = '/Users/dominicburrows/Dropbox/PhD/Analysis/my_scripts/GitHub/'
l_data = '/Users/dominicburrows/Dropbox/PhD/analysis/Project/'
l_fig = '/Users/dominicburrows/Dropbox/PhD/figures/'

s_code = '/cndd3/dburrows/CODE/'
s_data = '/cndd3/dburrows/DATA/'
s_fig = '/cndd3/dburrows/FIGS/'

%load_ext autoreload
sys.version

'3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:23:11) \n[GCC 9.4.0]'

# Estimate splice junctions from SMARTseq scRNA

In [2]:
#load in metadata 
os.chdir('/cndd3/dburrows/DATA/public_datasets/splice//smartseq_mouse_p53-59_MOp_pachter21')
meta = sc.read_h5ad('isoform.h5ad')

/home/dburrows/mambaforge/envs/base_conda/lib/python3.9/site-packages/anndata/compat/__init__.py:229: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/home/dburrows/mambaforge/envs/base_conda/lib/python3.9/site-packages/anndata/compat/__init__.py:229: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


In [5]:
prac

AnnData object with n_obs × n_vars = 6160 × 69172
    obs: 'cluster_id', 'cluster_label', 'subclass_label', 'class_label', 'cluster_color', 'size', 'cell_id', 'subclass_id', 'class_id', 'cell_counts', 'leiden', 'louvain'
    var: 'transcript_id', 'gene_id', 'gene_name', 'transcript_name', 'length', 'gene_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'leiden', 'louvain', 'neighbors'
    obsm: 'X_nca', 'X_nca_tsne', 'X_pca', 'X_pca_tsne'
    layers: 'X', 'log1p', 'norm'
    obsp: 'distances', 'connectivities'

In [16]:
print(prac.obs['class_label'].unique())
print(prac.obs['subclass_label'].unique())
print(prac.obs['cell_id'])

['GABAergic', 'Glutamatergic', 'Non-Neuronal', 'Low Quality']
Categories (4, object): ['GABAergic', 'Glutamatergic', 'Low Quality', 'Non-Neuronal']
['Vip', 'L5 IT', 'Lamp5', 'Sst', 'L5/6 NP', ..., 'Low Quality', 'L6 IT Car3', 'SMC', 'VLMC', 'Endo']
Length: 18
Categories (18, object): ['Astro', 'Endo', 'L2/3 IT', 'L5 PT', ..., 'Sncg', 'Sst', 'VLMC', 'Vip']
0        LS-15395_S47_E1-50
1        LS-15395_S48_E1-50
2        LS-15395_S49_E1-50
3        LS-15395_S50_E1-50
4        LS-15395_S51_E1-50
               ...         
6575    SM-GE935_S380_E1-50
6576    SM-GE935_S381_E1-50
6577    SM-GE935_S382_E1-50
6578    SM-GE935_S383_E1-50
6579    SM-GE935_S384_E1-50
Name: cell_id, Length: 6160, dtype: object


In [24]:
prac.var['transcript_id']

Xkr4-203_ENSMUST00000162897              ENSMUST00000162897
Xkr4-202_ENSMUST00000159265              ENSMUST00000159265
Xkr4-201_ENSMUST00000070533              ENSMUST00000070533
Gm1992-201_ENSMUST00000161581            ENSMUST00000161581
Gm37381-201_ENSMUST00000192427           ENSMUST00000192427
                                                ...        
AC168977.1-201_ENSMUST00000115924        ENSMUST00000115924
AC149090.1-201_ENSMUST00000179505        ENSMUST00000179505
AC149090.1-202_ENSMUST00000178343        ENSMUST00000178343
CAAA01118383.1-201_ENSMUST00000077235    ENSMUST00000077235
CAAA01147332.1-201_ENSMUST00000179436    ENSMUST00000179436
Name: transcript_id, Length: 69172, dtype: object

In [23]:
prac.X

array([[-0.28210094, -0.35316861, -0.57561437, ..., -0.33196362,
        -0.88440494, -2.57588301],
       [-0.28210094, -0.35316861,  1.37341428, ...,  0.0444962 ,
         0.38215863,  0.35461425],
       [-0.28210094, -0.35316861,  2.46693285, ...,  0.30307361,
        -0.88440494, -0.00340472],
       ...,
       [-0.28210094, -0.35316861, -0.57561437, ..., -0.64143172,
        -0.88440494,  0.76189187],
       [-0.28210094, -0.35316861, -0.57561437, ...,  1.41658933,
        -0.88440494, -2.57588301],
       [-0.28210094, -0.35316861, -0.57561437, ..., -0.37447894,
         1.85345286,  0.85364335]])

In [24]:
os.chdir('/cndd/Public_Datasets/BICCN/BICCN_minibrain_data/data_freeze/smarter_cells')
exon = pd.read_csv('exon.counts.csv.gz')
exon

,Unnamed: 0,LS-15395_S41_E1-50,LS-15395_S42_E1-50,LS-15395_S43_E1-50,LS-15395_S44_E1-50,LS-15395_S46_E1-50,LS-15395_S47_E1-50,LS-15395_S48_E1-50,LS-15395_S49_E1-50,LS-15395_S50_E1-50,...,SM-GE92M_S087_E1-50,SM-GE92M_S088_E1-50,SM-GE92M_S089_E1-50,SM-GE92M_S090_E1-50,SM-GE92M_S091_E1-50,SM-GE92M_S092_E1-50,SM-GE92M_S093_E1-50,SM-GE92M_S094_E1-50,SM-GE92M_S095_E1-50,SM-GE92M_S096_E1-50
0,0610005C13Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0610006L08Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0610007P14Rik,76,177,109,214,193,239,123,84,134,...,69,29,1,67,42,63,79,77,20,66
3,0610009B22Rik,188,224,67,59,174,413,82,127,177,...,90,26,22,57,48,66,63,92,87,0
4,0610009E02Rik,18,0,0,0,2,0,0,0,2,...,0,0,3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45763,n-R5s142,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45764,n-R5s143,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45765,n-R5s144,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45766,n-R5s146,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
os.chdir('/cndd3/dburrows/DATA/public_datasets/splice/smartseq_mouse_p53-59_MOp_pachter21/')
meta =  pd.read_csv('sample_metadata.csv.gz')
exon = pd.read_csv('exon.counts.csv.gz')
intron = pd.read_csv('intron.counts.csv.gz')

cluster.annotation.csv	exon.counts.csv.gz    QC.csv
cluster.membership.csv	intron.counts.csv.gz  sample_metadata.csv.gz


In [30]:
meta

,Unnamed: 0,ar_id,exp_component_vendor_name,batch,batch_vendor_name,tube,tube_internal_name,tube_contents_nm,tube_contents_nm_from_vendor,tube_avg_size_bp,...,Injection_type,NewBatch,Vendor,MultiPlex,percent_reads_aligned_to_rrna_rmsk,percent_reads_aligned_to_trna_rmsk,percent_reads_aligned_to_ncrna,percent_reads_aligned_to_introns,percent_reads_aligned_to_intergenic,percent_reads_aligned_unique
0,LS-15395_S41_E1-50,543183476,LS-15395-41,R8S4-160812,RSC-035,LS-15395,R8S4-16081206,4.619923,7.578116,529,...,0.0,HistoricalData,Covance,index96,NaN,NaN,15.162980,9.855384,3.264771,84.993663
1,LS-15395_S42_E1-50,543183474,LS-15395-42,R8S4-160812,RSC-035,LS-15395,R8S4-16081206,4.619923,7.578116,529,...,0.0,HistoricalData,Covance,index96,NaN,NaN,16.281620,6.763949,2.856108,87.252036
2,LS-15395_S43_E1-50,543183472,LS-15395-43,R8S4-160812,RSC-035,LS-15395,R8S4-16081206,4.619923,7.578116,529,...,0.0,HistoricalData,Covance,index96,NaN,NaN,14.448787,13.283785,4.473082,84.439773
3,LS-15395_S44_E1-50,543183470,LS-15395-44,R8S4-160812,RSC-035,LS-15395,R8S4-16081206,4.619923,7.578116,529,...,0.0,HistoricalData,Covance,index96,NaN,NaN,16.687114,7.724415,3.083142,87.400895
4,LS-15395_S46_E1-50,543183466,LS-15395-46,R8S4-160812,RSC-035,LS-15395,R8S4-16081206,4.619923,7.578116,529,...,0.0,HistoricalData,Covance,index96,NaN,NaN,15.844740,8.100115,3.478772,85.846808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6295,SM-GE92M_S092_E1-50,708586022,SM-GE92M-S092,R8S4-180530,RSC-157,SM-GE92M,R8S4-18053006,3.249174,1.950000,392,...,NaN,NewData,Broad,index384,0.692675,0.054178,4.007977,12.591947,27.800393,NaN
6296,SM-GE92M_S093_E1-50,708586372,SM-GE92M-S093,R8S4-180530,RSC-157,SM-GE92M,R8S4-18053006,3.249174,1.950000,392,...,NaN,NewData,Broad,index384,0.409496,0.051422,3.830221,9.094367,29.950267,NaN
6297,SM-GE92M_S094_E1-50,708586078,SM-GE92M-S094,R8S4-180530,RSC-157,SM-GE92M,R8S4-18053006,3.249174,1.950000,392,...,NaN,NewData,Broad,index384,0.610336,0.086938,3.419039,9.457431,29.709904,NaN
6298,SM-GE92M_S095_E1-50,708586242,SM-GE92M-S095,R8S4-180530,RSC-157,SM-GE92M,R8S4-18053006,3.249174,1.950000,392,...,NaN,NewData,Broad,index384,0.403770,0.040953,2.808983,6.679797,24.221576,NaN


In [21]:
exon

,Unnamed: 0,LS-15395_S41_E1-50,LS-15395_S42_E1-50,LS-15395_S43_E1-50,LS-15395_S44_E1-50,LS-15395_S46_E1-50,LS-15395_S47_E1-50,LS-15395_S48_E1-50,LS-15395_S49_E1-50,LS-15395_S50_E1-50,...,SM-GE92M_S087_E1-50,SM-GE92M_S088_E1-50,SM-GE92M_S089_E1-50,SM-GE92M_S090_E1-50,SM-GE92M_S091_E1-50,SM-GE92M_S092_E1-50,SM-GE92M_S093_E1-50,SM-GE92M_S094_E1-50,SM-GE92M_S095_E1-50,SM-GE92M_S096_E1-50
0,0610005C13Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0610006L08Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0610007P14Rik,76,177,109,214,193,239,123,84,134,...,69,29,1,67,42,63,79,77,20,66
3,0610009B22Rik,188,224,67,59,174,413,82,127,177,...,90,26,22,57,48,66,63,92,87,0
4,0610009E02Rik,18,0,0,0,2,0,0,0,2,...,0,0,3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45763,n-R5s142,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45764,n-R5s143,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45765,n-R5s144,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45766,n-R5s146,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
intron

,Unnamed: 0,LS-15395_S41_E1-50,LS-15395_S42_E1-50,LS-15395_S43_E1-50,LS-15395_S44_E1-50,LS-15395_S46_E1-50,LS-15395_S47_E1-50,LS-15395_S48_E1-50,LS-15395_S49_E1-50,LS-15395_S50_E1-50,...,SM-GE92M_S087_E1-50,SM-GE92M_S088_E1-50,SM-GE92M_S089_E1-50,SM-GE92M_S090_E1-50,SM-GE92M_S091_E1-50,SM-GE92M_S092_E1-50,SM-GE92M_S093_E1-50,SM-GE92M_S094_E1-50,SM-GE92M_S095_E1-50,SM-GE92M_S096_E1-50
0,0610005C13Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1,0610006L08Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0610007P14Rik,0,0,0,0,18,0,0,0,12,...,0,0,38,0,1,0,0,0,0,0
3,0610009B22Rik,0,0,0,1,0,0,0,0,0,...,1,3,0,6,2,9,3,1,1,0
4,0610009E02Rik,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45763,n-R5s142,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45764,n-R5s143,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45765,n-R5s144,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45766,n-R5s146,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
intron.columns

In [20]:
#meta =  pd.read_csv('sample_metadata.csv.gz')
np.unique(np.asarray([i.split('-')[0] for i in meta['cell_name']]))


array(['Ctgf', 'Nr5a1', 'Pvalb', 'Rbp4', 'Scnn1a', 'Slc17a6', 'Slc17a7',
       'Slc32a1', 'Snap25'], dtype='<U7')

In [13]:
for i in meta.columns: 
    if 'cell' in i: print(i)


patched_cell_container
cell_name
cell_id
cell_prep_type


In [21]:
sum(prac['Unnamed: 0'] == 'Oxr1-204')

0

In [22]:
sum(prac['Unnamed: 0'] == '202')

1

In [23]:
prac['Unnamed: 0'] [prac['Unnamed: 0'].str.contains('202')]

9556          Gm12020
9557          Gm12021
9558          Gm12022
9559          Gm12024
9560          Gm12026
9643          Gm12202
10078         Gm13202
11308         Gm16202
11540         Gm17202
13009         Gm19202
13277         Gm20203
13278         Gm20219
13279          Gm2022
13280         Gm20223
13281         Gm20236
13282         Gm20239
13283         Gm20241
13284         Gm20257
13285         Gm20267
13286         Gm20268
13287          Gm2027
13288         Gm20275
13289         Gm20277
13290         Gm20287
13291         Gm20294
15471         Gm31202
16093         Gm32020
16094         Gm32022
16095         Gm32023
16096         Gm32024
16097         Gm32025
16098         Gm32026
16099         Gm32027
16100         Gm32028
16101         Gm32029
16214         Gm32202
17001         Gm33202
19312         Gm36202
20029          Gm4202
22420          Gm8202
22996          Gm9202
24506         Ifi202b
26886    LOC102641202
26967    LOC102642202
27645    LOC105243202
28431    L

In [28]:
prac['Unnamed: 0'] [prac['Unnamed: 0'].str.contains('Pv')]

38358       Pvalb
38359         Pvr
38360    Pvrig-ps
38361       Pvrl1
38362       Pvrl2
38363       Pvrl3
38364       Pvrl4
38365        Pvt1
Name: Unnamed: 0, dtype: object

In [31]:
prac['Unnamed: 0'] [prac['Unnamed: 0'].str.contains('201')].values

array(['2010001A14Rik', '2010001E11Rik', '2010001M07Rik', '2010002M12Rik',
       '2010003K11Rik', '2010005H15Rik', '2010009K17Rik', '2010010A06Rik',
       '2010012O05Rik', '2010016I18Rik', '2010106C02Rik', '2010106E10Rik',
       '2010107E04Rik', '2010107G12Rik', '2010107G23Rik', '2010109A12Rik',
       '2010109I03Rik', '2010110K18Rik', '2010111I01Rik', '2010203P06Rik',
       '2010204K13Rik', '2010300C02Rik', '2010300F17Rik', '2010308F09Rik',
       '2010310C07Rik', '2010315B03Rik', '2010320M18Rik', '2010320O07Rik',
       '9330162012Rik', 'AW112010', 'E130201H02Rik', 'Gm11201', 'Gm12010',
       'Gm12011', 'Gm12012', 'Gm12014', 'Gm12015', 'Gm12017', 'Gm12018',
       'Gm12019', 'Gm12201', 'Gm13201', 'Gm15201', 'Gm18201', 'Gm19201',
       'Gm20104', 'Gm2011', 'Gm20110', 'Gm2012', 'Gm20121', 'Gm20124',
       'Gm20125', 'Gm20128', 'Gm20136', 'Gm20139', 'Gm20147', 'Gm20149',
       'Gm20150', 'Gm20153', 'Gm20156', 'Gm20159', 'Gm2016', 'Gm20161',
       'Gm20162', 'Gm20164', 'Gm20172'

In [5]:
prac

,Unnamed: 0,LS-15395_S41_E1-50,LS-15395_S42_E1-50,LS-15395_S43_E1-50,LS-15395_S44_E1-50,LS-15395_S46_E1-50,LS-15395_S47_E1-50,LS-15395_S48_E1-50,LS-15395_S49_E1-50,LS-15395_S50_E1-50,...,SM-GE92M_S087_E1-50,SM-GE92M_S088_E1-50,SM-GE92M_S089_E1-50,SM-GE92M_S090_E1-50,SM-GE92M_S091_E1-50,SM-GE92M_S092_E1-50,SM-GE92M_S093_E1-50,SM-GE92M_S094_E1-50,SM-GE92M_S095_E1-50,SM-GE92M_S096_E1-50
0,0610005C13Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0610006L08Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0610007P14Rik,76,177,109,214,193,239,123,84,134,...,69,29,1,67,42,63,79,77,20,66
3,0610009B22Rik,188,224,67,59,174,413,82,127,177,...,90,26,22,57,48,66,63,92,87,0
4,0610009E02Rik,18,0,0,0,2,0,0,0,2,...,0,0,3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45763,n-R5s142,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45764,n-R5s143,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45765,n-R5s144,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45766,n-R5s146,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
ok = np.sum(prac,axis=0)

In [17]:
np.min(prac[prac!=0])

/home/dburrows/mambaforge/envs/base_conda/lib/python3.9/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)


Unnamed: 0             0610005C13Rik
LS-15395_S41_E1-50               1.0
LS-15395_S42_E1-50               1.0
LS-15395_S43_E1-50               1.0
LS-15395_S44_E1-50               1.0
                           ...      
SM-GE92M_S092_E1-50              1.0
SM-GE92M_S093_E1-50              1.0
SM-GE92M_S094_E1-50              1.0
SM-GE92M_S095_E1-50              1.0
SM-GE92M_S096_E1-50              1.0
Length: 6301, dtype: object

In [18]:
2**0

1

In [35]:
%%bash
cd $CODE3/deepNN_splice
git add **
git commit -m 'pachtor data'
git push

[main 793af86] pachtor data
 2 files changed, 2406 insertions(+), 222 deletions(-)
 create mode 100644 data_process.ipynb
 rewrite workspace.ipynb (77%)


fatal: could not read Username for 'https://github.com': No such device or address


CalledProcessError: Command 'b"cd $CODE3/deepNN_splice\ngit add **\ngit commit -m 'pachtor data'\ngit push\n"' returned non-zero exit status 128.

In [32]:
prac

,Unnamed: 0,LS-15395_S41_E1-50,LS-15395_S42_E1-50,LS-15395_S43_E1-50,LS-15395_S44_E1-50,LS-15395_S46_E1-50,LS-15395_S47_E1-50,LS-15395_S48_E1-50,LS-15395_S49_E1-50,LS-15395_S50_E1-50,...,SM-GE92M_S087_E1-50,SM-GE92M_S088_E1-50,SM-GE92M_S089_E1-50,SM-GE92M_S090_E1-50,SM-GE92M_S091_E1-50,SM-GE92M_S092_E1-50,SM-GE92M_S093_E1-50,SM-GE92M_S094_E1-50,SM-GE92M_S095_E1-50,SM-GE92M_S096_E1-50
0,0610005C13Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1,0610006L08Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0610007P14Rik,0,0,0,0,18,0,0,0,12,...,0,0,38,0,1,0,0,0,0,0
3,0610009B22Rik,0,0,0,1,0,0,0,0,0,...,1,3,0,6,2,9,3,1,1,0
4,0610009E02Rik,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45763,n-R5s142,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45764,n-R5s143,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45765,n-R5s144,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45766,n-R5s146,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
